# Speech Understanding - Programming Assignment 3

    Aryan Tiwari B20AI056

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import sys
import torch
import librosa
import torchaudio

import torchmetrics

# import gradio as gr
import wandb as wb


## Question 1

**Goal:** 

    The task is to classify the audio samples into Real and Fake

**Tasks:** 

    —- Use the SSL W2V model trained for LA and DF tracks of the ASVSpoof dataset.

    —- Download the custom dataset from here. Report the AUC and EER on this dataset. 
    
    —- Analyze the performance of the model.
    
    —- Finetune the model on FOR dataset. 
    
    —- Report the performance using AUC and EER on For dataset. 
    
    —- Use the model trained on the FOR dataset to evaluate the custom dataset. Report the EER and AUC
    
    —- Comment on the change in performance, if any. 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
!git clone https://github.com/TakHemlata/SSL_Anti-spoofing.git

In [ ]:
import random
from typing import Union

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
import fairseq


############################
## FOR fine-tuned SSL MODEL
############################


class SSLModel(nn.Module):
    def __init__(self,device):
        super(SSLModel, self).__init__()
        
        cp_path = 'xlsr2_300m.pt'   # Change the pre-trained XLSR model path. 
        model, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([cp_path])
        self.model = model[0]
        self.device=device
        self.out_dim = 1024
        return

    def extract_feat(self, input_data):
        
        # put the model to GPU if it not there
        if next(self.model.parameters()).device != input_data.device \
           or next(self.model.parameters()).dtype != input_data.dtype:
            self.model.to(input_data.device, dtype=input_data.dtype)
            self.model.train()

        
        if True:
            # input should be in shape (batch, length)
            if input_data.ndim == 3:
                input_tmp = input_data[:, :, 0]
            else:
                input_tmp = input_data
                
            # [batch, length, dim]
            emb = self.model(input_tmp, mask=False, features_only=True)['x']
        return emb


#---------AASIST back-end------------------------#
''' Jee-weon Jung, Hee-Soo Heo, Hemlata Tak, Hye-jin Shim, Joon Son Chung, Bong-Jin Lee, Ha-Jin Yu and Nicholas Evans. 
    AASIST: Audio Anti-Spoofing Using Integrated Spectro-Temporal Graph Attention Networks. 
    In Proc. ICASSP 2022, pp: 6367--6371.'''


class GraphAttentionLayer(nn.Module):
    def __init__(self, in_dim, out_dim, **kwargs):
        super().__init__()

        # attention map
        self.att_proj = nn.Linear(in_dim, out_dim)
        self.att_weight = self._init_new_params(out_dim, 1)

        # project
        self.proj_with_att = nn.Linear(in_dim, out_dim)
        self.proj_without_att = nn.Linear(in_dim, out_dim)

        # batch norm
        self.bn = nn.BatchNorm1d(out_dim)

        # dropout for inputs
        self.input_drop = nn.Dropout(p=0.2)

        # activate
        self.act = nn.SELU(inplace=True)

        # temperature
        self.temp = 1.
        if "temperature" in kwargs:
            self.temp = kwargs["temperature"]

    def forward(self, x):
        '''
        x   :(#bs, #node, #dim)
        '''
        # apply input dropout
        x = self.input_drop(x)

        # derive attention map
        att_map = self._derive_att_map(x)

        # projection
        x = self._project(x, att_map)

        # apply batch norm
        x = self._apply_BN(x)
        x = self.act(x)
        return x

    def _pairwise_mul_nodes(self, x):
        '''
        Calculates pairwise multiplication of nodes.
        - for attention map
        x           :(#bs, #node, #dim)
        out_shape   :(#bs, #node, #node, #dim)
        '''

        nb_nodes = x.size(1)
        x = x.unsqueeze(2).expand(-1, -1, nb_nodes, -1)
        x_mirror = x.transpose(1, 2)

        return x * x_mirror

    def _derive_att_map(self, x):
        '''
        x           :(#bs, #node, #dim)
        out_shape   :(#bs, #node, #node, 1)
        '''
        att_map = self._pairwise_mul_nodes(x)
        # size: (#bs, #node, #node, #dim_out)
        att_map = torch.tanh(self.att_proj(att_map))
        # size: (#bs, #node, #node, 1)
        att_map = torch.matmul(att_map, self.att_weight)

        # apply temperature
        att_map = att_map / self.temp

        att_map = F.softmax(att_map, dim=-2)

        return att_map

    def _project(self, x, att_map):
        x1 = self.proj_with_att(torch.matmul(att_map.squeeze(-1), x))
        x2 = self.proj_without_att(x)

        return x1 + x2

    def _apply_BN(self, x):
        org_size = x.size()
        x = x.view(-1, org_size[-1])
        x = self.bn(x)
        x = x.view(org_size)

        return x

    def _init_new_params(self, *size):
        out = nn.Parameter(torch.FloatTensor(*size))
        nn.init.xavier_normal_(out)
        return out


class HtrgGraphAttentionLayer(nn.Module):
    def __init__(self, in_dim, out_dim, **kwargs):
        super().__init__()

        self.proj_type1 = nn.Linear(in_dim, in_dim)
        self.proj_type2 = nn.Linear(in_dim, in_dim)

        # attention map
        self.att_proj = nn.Linear(in_dim, out_dim)
        self.att_projM = nn.Linear(in_dim, out_dim)

        self.att_weight11 = self._init_new_params(out_dim, 1)
        self.att_weight22 = self._init_new_params(out_dim, 1)
        self.att_weight12 = self._init_new_params(out_dim, 1)
        self.att_weightM = self._init_new_params(out_dim, 1)

        # project
        self.proj_with_att = nn.Linear(in_dim, out_dim)
        self.proj_without_att = nn.Linear(in_dim, out_dim)

        self.proj_with_attM = nn.Linear(in_dim, out_dim)
        self.proj_without_attM = nn.Linear(in_dim, out_dim)

        # batch norm
        self.bn = nn.BatchNorm1d(out_dim)

        # dropout for inputs
        self.input_drop = nn.Dropout(p=0.2)

        # activate
        self.act = nn.SELU(inplace=True)

        # temperature
        self.temp = 1.
        if "temperature" in kwargs:
            self.temp = kwargs["temperature"]

    def forward(self, x1, x2, master=None):
        '''
        x1  :(#bs, #node, #dim)
        x2  :(#bs, #node, #dim)
        '''
        #print('x1',x1.shape)
        #print('x2',x2.shape)
        num_type1 = x1.size(1)
        num_type2 = x2.size(1)
        #print('num_type1',num_type1)
        #print('num_type2',num_type2)
        x1 = self.proj_type1(x1)
        #print('proj_type1',x1.shape)
        x2 = self.proj_type2(x2)
        #print('proj_type2',x2.shape)
        x = torch.cat([x1, x2], dim=1)
        #print('Concat x1 and x2',x.shape)
        
        if master is None:
            master = torch.mean(x, dim=1, keepdim=True)
            #print('master',master.shape)
        # apply input dropout
        x = self.input_drop(x)

        # derive attention map
        att_map = self._derive_att_map(x, num_type1, num_type2)
        #print('master',master.shape)
        # directional edge for master node
        master = self._update_master(x, master)
        #print('master',master.shape)
        # projection
        x = self._project(x, att_map)
        #print('proj x',x.shape)
        # apply batch norm
        x = self._apply_BN(x)
        x = self.act(x)

        x1 = x.narrow(1, 0, num_type1)
        #print('x1',x1.shape)
        x2 = x.narrow(1, num_type1, num_type2)
        #print('x2',x2.shape)
        return x1, x2, master

    def _update_master(self, x, master):

        att_map = self._derive_att_map_master(x, master)
        master = self._project_master(x, master, att_map)

        return master

    def _pairwise_mul_nodes(self, x):
        '''
        Calculates pairwise multiplication of nodes.
        - for attention map
        x           :(#bs, #node, #dim)
        out_shape   :(#bs, #node, #node, #dim)
        '''

        nb_nodes = x.size(1)
        x = x.unsqueeze(2).expand(-1, -1, nb_nodes, -1)
        x_mirror = x.transpose(1, 2)

        return x * x_mirror

    def _derive_att_map_master(self, x, master):
        '''
        x           :(#bs, #node, #dim)
        out_shape   :(#bs, #node, #node, 1)
        '''
        att_map = x * master
        att_map = torch.tanh(self.att_projM(att_map))

        att_map = torch.matmul(att_map, self.att_weightM)

        # apply temperature
        att_map = att_map / self.temp

        att_map = F.softmax(att_map, dim=-2)

        return att_map

    def _derive_att_map(self, x, num_type1, num_type2):
        '''
        x           :(#bs, #node, #dim)
        out_shape   :(#bs, #node, #node, 1)
        '''
        att_map = self._pairwise_mul_nodes(x)
        # size: (#bs, #node, #node, #dim_out)
        att_map = torch.tanh(self.att_proj(att_map))
        # size: (#bs, #node, #node, 1)

        att_board = torch.zeros_like(att_map[:, :, :, 0]).unsqueeze(-1)

        att_board[:, :num_type1, :num_type1, :] = torch.matmul(
            att_map[:, :num_type1, :num_type1, :], self.att_weight11)
        att_board[:, num_type1:, num_type1:, :] = torch.matmul(
            att_map[:, num_type1:, num_type1:, :], self.att_weight22)
        att_board[:, :num_type1, num_type1:, :] = torch.matmul(
            att_map[:, :num_type1, num_type1:, :], self.att_weight12)
        att_board[:, num_type1:, :num_type1, :] = torch.matmul(
            att_map[:, num_type1:, :num_type1, :], self.att_weight12)

        att_map = att_board

        

        # apply temperature
        att_map = att_map / self.temp

        att_map = F.softmax(att_map, dim=-2)

        return att_map

    def _project(self, x, att_map):
        x1 = self.proj_with_att(torch.matmul(att_map.squeeze(-1), x))
        x2 = self.proj_without_att(x)

        return x1 + x2

    def _project_master(self, x, master, att_map):

        x1 = self.proj_with_attM(torch.matmul(
            att_map.squeeze(-1).unsqueeze(1), x))
        x2 = self.proj_without_attM(master)

        return x1 + x2

    def _apply_BN(self, x):
        org_size = x.size()
        x = x.view(-1, org_size[-1])
        x = self.bn(x)
        x = x.view(org_size)

        return x

    def _init_new_params(self, *size):
        out = nn.Parameter(torch.FloatTensor(*size))
        nn.init.xavier_normal_(out)
        return out


class GraphPool(nn.Module):
    def __init__(self, k: float, in_dim: int, p: Union[float, int]):
        super().__init__()
        self.k = k
        self.sigmoid = nn.Sigmoid()
        self.proj = nn.Linear(in_dim, 1)
        self.drop = nn.Dropout(p=p) if p > 0 else nn.Identity()
        self.in_dim = in_dim

    def forward(self, h):
        Z = self.drop(h)
        weights = self.proj(Z)
        scores = self.sigmoid(weights)
        new_h = self.top_k_graph(scores, h, self.k)

        return new_h

    def top_k_graph(self, scores, h, k):
        """
        args
        =====
        scores: attention-based weights (#bs, #node, 1)
        h: graph data (#bs, #node, #dim)
        k: ratio of remaining nodes, (float)
        returns
        =====
        h: graph pool applied data (#bs, #node', #dim)
        """
        _, n_nodes, n_feat = h.size()
        n_nodes = max(int(n_nodes * k), 1)
        _, idx = torch.topk(scores, n_nodes, dim=1)
        idx = idx.expand(-1, -1, n_feat)

        h = h * scores
        h = torch.gather(h, 1, idx)

        return h




class Residual_block(nn.Module):
    def __init__(self, nb_filts, first=False):
        super().__init__()
        self.first = first

        if not self.first:
            self.bn1 = nn.BatchNorm2d(num_features=nb_filts[0])
        self.conv1 = nn.Conv2d(in_channels=nb_filts[0],
                               out_channels=nb_filts[1],
                               kernel_size=(2, 3),
                               padding=(1, 1),
                               stride=1)
        self.selu = nn.SELU(inplace=True)

        self.bn2 = nn.BatchNorm2d(num_features=nb_filts[1])
        self.conv2 = nn.Conv2d(in_channels=nb_filts[1],
                               out_channels=nb_filts[1],
                               kernel_size=(2, 3),
                               padding=(0, 1),
                               stride=1)

        if nb_filts[0] != nb_filts[1]:
            self.downsample = True
            self.conv_downsample = nn.Conv2d(in_channels=nb_filts[0],
                                             out_channels=nb_filts[1],
                                             padding=(0, 1),
                                             kernel_size=(1, 3),
                                             stride=1)

        else:
            self.downsample = False
        

    def forward(self, x):
        identity = x
        if not self.first:
            out = self.bn1(x)
            out = self.selu(out)
        else:
            out = x

        #print('out',out.shape)
        out = self.conv1(x)

        #print('aft conv1 out',out.shape)
        out = self.bn2(out)
        out = self.selu(out)
        # print('out',out.shape)
        out = self.conv2(out)
        #print('conv2 out',out.shape)
        
        if self.downsample:
            identity = self.conv_downsample(identity)

        out += identity
        #out = self.mp(out)
        return out


class Model(nn.Module):
    def __init__(self, args,device):
        super().__init__()
        self.device = device
        
        # AASIST parameters
        filts = [128, [1, 32], [32, 32], [32, 64], [64, 64]]
        gat_dims = [64, 32]
        pool_ratios = [0.5, 0.5, 0.5, 0.5]
        temperatures =  [2.0, 2.0, 100.0, 100.0]


        ####
        # create network wav2vec 2.0
        ####
        self.ssl_model = SSLModel(self.device)
        self.LL = nn.Linear(self.ssl_model.out_dim, 128)

        self.first_bn = nn.BatchNorm2d(num_features=1)
        self.first_bn1 = nn.BatchNorm2d(num_features=64)
        self.drop = nn.Dropout(0.5, inplace=True)
        self.drop_way = nn.Dropout(0.2, inplace=True)
        self.selu = nn.SELU(inplace=True)

        # RawNet2 encoder
        self.encoder = nn.Sequential(
            nn.Sequential(Residual_block(nb_filts=filts[1], first=True)),
            nn.Sequential(Residual_block(nb_filts=filts[2])),
            nn.Sequential(Residual_block(nb_filts=filts[3])),
            nn.Sequential(Residual_block(nb_filts=filts[4])),
            nn.Sequential(Residual_block(nb_filts=filts[4])),
            nn.Sequential(Residual_block(nb_filts=filts[4])))

        self.attention = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=(1,1)),
            nn.SELU(inplace=True),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 64, kernel_size=(1,1)),
            
        )
        # position encoding
        self.pos_S = nn.Parameter(torch.randn(1, 42, filts[-1][-1]))
        
        self.master1 = nn.Parameter(torch.randn(1, 1, gat_dims[0]))
        self.master2 = nn.Parameter(torch.randn(1, 1, gat_dims[0]))
        
        # Graph module
        self.GAT_layer_S = GraphAttentionLayer(filts[-1][-1],
                                               gat_dims[0],
                                               temperature=temperatures[0])
        self.GAT_layer_T = GraphAttentionLayer(filts[-1][-1],
                                               gat_dims[0],
                                               temperature=temperatures[1])
        # HS-GAL layer 
        self.HtrgGAT_layer_ST11 = HtrgGraphAttentionLayer(
            gat_dims[0], gat_dims[1], temperature=temperatures[2])
        self.HtrgGAT_layer_ST12 = HtrgGraphAttentionLayer(
            gat_dims[1], gat_dims[1], temperature=temperatures[2])
        self.HtrgGAT_layer_ST21 = HtrgGraphAttentionLayer(
            gat_dims[0], gat_dims[1], temperature=temperatures[2])
        self.HtrgGAT_layer_ST22 = HtrgGraphAttentionLayer(
            gat_dims[1], gat_dims[1], temperature=temperatures[2])

        # Graph pooling layers
        self.pool_S = GraphPool(pool_ratios[0], gat_dims[0], 0.3)
        self.pool_T = GraphPool(pool_ratios[1], gat_dims[0], 0.3)
        self.pool_hS1 = GraphPool(pool_ratios[2], gat_dims[1], 0.3)
        self.pool_hT1 = GraphPool(pool_ratios[2], gat_dims[1], 0.3)

        self.pool_hS2 = GraphPool(pool_ratios[2], gat_dims[1], 0.3)
        self.pool_hT2 = GraphPool(pool_ratios[2], gat_dims[1], 0.3)
        
        self.out_layer = nn.Linear(5 * gat_dims[1], 2)

    def forward(self, x):
        #-------pre-trained Wav2vec model fine tunning ------------------------##
        x_ssl_feat = self.ssl_model.extract_feat(x.squeeze(-1))
        x = self.LL(x_ssl_feat) #(bs,frame_number,feat_out_dim)
        
        # post-processing on front-end features
        x = x.transpose(1, 2)   #(bs,feat_out_dim,frame_number)
        x = x.unsqueeze(dim=1) # add channel 
        x = F.max_pool2d(x, (3, 3))
        x = self.first_bn(x)
        x = self.selu(x)

        # RawNet2-based encoder
        x = self.encoder(x)
        x = self.first_bn1(x)
        x = self.selu(x)
        
        w = self.attention(x)
        
        #------------SA for spectral feature-------------#
        w1 = F.softmax(w,dim=-1)
        m = torch.sum(x * w1, dim=-1)
        e_S = m.transpose(1, 2) + self.pos_S 
        
        # graph module layer
        gat_S = self.GAT_layer_S(e_S)
        out_S = self.pool_S(gat_S)  # (#bs, #node, #dim)
        
        #------------SA for temporal feature-------------#
        w2 = F.softmax(w,dim=-2)
        m1 = torch.sum(x * w2, dim=-2)
     
        e_T = m1.transpose(1, 2)
       
        # graph module layer
        gat_T = self.GAT_layer_T(e_T)
        out_T = self.pool_T(gat_T)
        
        # learnable master node
        master1 = self.master1.expand(x.size(0), -1, -1)
        master2 = self.master2.expand(x.size(0), -1, -1)

        # inference 1
        out_T1, out_S1, master1 = self.HtrgGAT_layer_ST11(
            out_T, out_S, master=self.master1)

        out_S1 = self.pool_hS1(out_S1)
        out_T1 = self.pool_hT1(out_T1)

        out_T_aug, out_S_aug, master_aug = self.HtrgGAT_layer_ST12(
            out_T1, out_S1, master=master1)
        out_T1 = out_T1 + out_T_aug
        out_S1 = out_S1 + out_S_aug
        master1 = master1 + master_aug

        # inference 2
        out_T2, out_S2, master2 = self.HtrgGAT_layer_ST21(
            out_T, out_S, master=self.master2)
        out_S2 = self.pool_hS2(out_S2)
        out_T2 = self.pool_hT2(out_T2)

        out_T_aug, out_S_aug, master_aug = self.HtrgGAT_layer_ST22(
            out_T2, out_S2, master=master2)
        out_T2 = out_T2 + out_T_aug
        out_S2 = out_S2 + out_S_aug
        master2 = master2 + master_aug

        out_T1 = self.drop_way(out_T1)
        out_T2 = self.drop_way(out_T2)
        out_S1 = self.drop_way(out_S1)
        out_S2 = self.drop_way(out_S2)
        master1 = self.drop_way(master1)
        master2 = self.drop_way(master2)

        out_T = torch.max(out_T1, out_T2)
        out_S = torch.max(out_S1, out_S2)
        master = torch.max(master1, master2)

        # Readout operation
        T_max, _ = torch.max(torch.abs(out_T), dim=1)
        T_avg = torch.mean(out_T, dim=1)

        S_max, _ = torch.max(torch.abs(out_S), dim=1)
        S_avg = torch.mean(out_S, dim=1)
        
        last_hidden = torch.cat(
            [T_max, T_avg, S_max, S_avg, master.squeeze(1)], dim=1)
        
        last_hidden = self.drop(last_hidden)
        output = self.out_layer(last_hidden)
        
        return output


In [ ]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import librosa
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import DatasetFolder
from torch import Tensor
import os
import json
from sklearn import metrics
from scipy.optimize import brentq
from scipy.interpolate import interp1d
from sklearn.metrics import roc_curve
from matplotlib import pyplot as plt

# Configuration
experiment_save_name = 'custom_subset_eval_1'
root_directory_path_rishabh_subset = '../Dataset_Speech_Assignment' 
batch_size = 8
device = 'cuda' if torch.cuda.is_available() else 'cpu'
args = None
random_seed = 42
torch.manual_seed(random_seed)

# Load the model
model = Model(args, 'cpu')

# Load the best state dict .pth file
state_dict_path = '../models/Best_LA_model_for_DF.pth'
state_dict = torch.load(state_dict_path, map_location='cpu')
for key in list(state_dict.keys()):
    state_dict[key.replace('module.', '')] = state_dict.pop(key)
model.load_state_dict(state_dict)


# Load the dataset
def pad(x, max_len=64600):
    x_len = x.shape[0]
    if x_len >= max_len:
        return x[:max_len]
    num_repeats = int(max_len / x_len) + 1
    padded_x = np.tile(x, (1, num_repeats))[:, :max_len][0]
    return padded_x


def loader_rishabhsubset(samplepath):
    cut = 64600
    X, fs = librosa.load(samplepath, sr=16000)
    X_pad = pad(X, cut)
    x_inp = Tensor(X_pad)
    return x_inp


dataset = DatasetFolder(root_directory_path_rishabh_subset, loader=loader_rishabhsubset, extensions=('wav', 'mp3', ))

# Load the dataloader
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define evaluation utilities
def compute_det_curve(target_scores, nontarget_scores):
    target_scores = np.array(target_scores)
    nontarget_scores = np.array(nontarget_scores)
    n_scores = target_scores.size + nontarget_scores.size
    all_scores = np.concatenate((target_scores, nontarget_scores))
    labels = np.concatenate((np.ones(target_scores.size), np.zeros(nontarget_scores.size)))
    indices = np.argsort(all_scores, kind='mergesort')
    labels = labels[indices]
    tar_trial_sums = np.cumsum(labels)
    nontarget_trial_sums = nontarget_scores.size - (np.arange(1, n_scores + 1) - tar_trial_sums)
    frr = np.concatenate((np.atleast_1d(0), tar_trial_sums / target_scores.size))
    far = np.concatenate((np.atleast_1d(1), nontarget_trial_sums / nontarget_scores.size))
    thresholds = np.concatenate((np.atleast_1d(all_scores[indices[0]] - 0.001), all_scores[indices]))
    return frr, far, thresholds

def compute_eer(target_scores, nontarget_scores):
    frr, far, thresholds = compute_det_curve(target_scores, nontarget_scores)
    abs_diffs = np.abs(frr - far)
    min_index = np.argmin(abs_diffs)
    eer = np.mean((frr[min_index], far[min_index]))
    return eer, thresholds[min_index]

def compute_eer_auc(target_scores, nontarget_scores):
    frr, far, thresholds = compute_det_curve(target_scores, nontarget_scores)
    abs_diffs = np.abs(frr - far)
    min_index = np.argmin(abs_diffs)
    eer = np.mean((frr[min_index], far[min_index]))
    auc = np.trapz(1 - frr, far)
    return eer, auc, thresholds[min_index]

# Evaluation script
def eval_script(model, loader, device, savepath='B20AI056_output', printlogs=True, savelogs=True, savefigs=True):
    with torch.no_grad():
        model = model.to(device)
        model.eval()
        scores = []
        truths = []
        for xs, labels in tqdm(loader):
            xs, labels = xs.to(device), labels.to(device)
            outputs = model(xs)
            scores.extend((outputs[:, 1]).data.cpu().numpy().ravel().tolist())
            truths.extend(labels.tolist())

        fpr, tpr, thresholds = metrics.roc_curve(np.array(truths), np.array(scores), pos_label=1)
        auc = metrics.auc(fpr, tpr)
        eer = brentq(lambda x: 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
        thresh = interp1d(fpr, thresholds)(eer)

        if printlogs:
            print('FINAL SCORES BELOW!!!!!!!!!!')
            print('eer:', eer, '        auc:', auc,  "    thresh:", thresh)
            print('FINAL SCORES ABOVE!!!!!!!!!!')
            print()
        
        if savelogs:
            with open(os.path.join(savepath, experiment_save_name + '.json'), 'w') as f:
                json.dump(
                    {
                        'eer': float(f'{eer}'),
                        'auc': float(f'{auc}'),
                        'thresh': float(f'{thresh}'),
                    }, f, indent=4
                )
            
        if savefigs:    
            # Plot ROC curve and save
            image_save_path = os.path.join(savepath, experiment_save_name + '.png')
            plt.figure()
            lw = 2
            plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % auc)
            plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title('Receiver Operating Characteristic (ROC) Curve')
            plt.legend(loc="lower right")
            plt.savefig(image_save_path)

    return eer, auc

eval_script(model, loader, device, printlogs=False, savelogs=False, savefigs=True)


In [ ]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import librosa
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import DatasetFolder
from torch import Tensor
import os
import json
from sklearn import metrics
from scipy.optimize import brentq
from scipy.interpolate import interp1d
from sklearn.metrics import roc_curve

from model import Model

###########################################
checkpoint_save_dir = 'checkpoint_save_dir'
experiment_save_name = 'train_eval_2_final'

# root_directory_path_rishabh_subset = '../Dataset_Speech_Assignment' # contains .wav and .mp3 files

root_directory_path_testing = '../Dataset_Speech_Assignment'  # contains .wav files only.
# root_directory_path_testing = '../for-2seconds/testing'  # contains .wav files only.
root_directory_path_train = '../for-2seconds/training'  # contains .wav files only.
root_directory_path_validation = '../for-2seconds/validation'  # contains .wav files only.

batch_size = 32
nepochs = 2
learningrate = 3e-4

device = 'cuda' if torch.cuda.is_available() else 'cpu'
args = None
random_seed = 42
torch.manual_seed(random_seed)
###########################################
# wandb

import wandb

wandb.init(
    project='supa3',
    
    config={
        'learning-rate': learningrate,
        'num-epochs': nepochs,
        'batch-size': batch_size,
        'random-seed': random_seed,
    }
)

###########################################




# load the model
model = Model(args, 'cpu')

# load the best state dict .pth file
state_dict_path = '../models/Best_LA_model_for_DF.pth'
state_dict = torch.load(state_dict_path, map_location='cpu')
for key in list(state_dict.keys()):
    state_dict[key.replace('module.', '')] = state_dict.pop(key)

model.load_state_dict(state_dict)


# load the dataset
def pad(x, max_len=64600):
    x_len = x.shape[0]
    if x_len >= max_len:
        return x[:max_len]
    # need to pad
    num_repeats = int(max_len / x_len)+1
    padded_x = np.tile(x, (1, num_repeats))[:, :max_len][0]
    return padded_x


def loader_rishabhsubset(samplepath):
    cut = 64600
    X, fs = librosa.load(samplepath, sr=16000)
    X_pad = pad(X, cut)
    x_inp = Tensor(X_pad)
    return x_inp


train_dataset = DatasetFolder(root_directory_path_train, loader=loader_rishabhsubset, extensions=('wav', 'mp3', ))
eval_dataset  = DatasetFolder(root_directory_path_validation, loader=loader_rishabhsubset, extensions=('wav', 'mp3', ))
test_dataset  = DatasetFolder(root_directory_path_testing, loader=loader_rishabhsubset, extensions=('wav', 'mp3', ))

# load the dataloader. 
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
eval_loader  = DataLoader(eval_dataset , batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset , batch_size=batch_size, shuffle=True)



#define the eval utils
def compute_det_curve(target_scores, nontarget_scores):
    target_scores = np.array(target_scores)
    nontarget_scores = np.array(nontarget_scores)
    n_scores = target_scores.size + nontarget_scores.size
    all_scores = np.concatenate((target_scores, nontarget_scores))
    labels = np.concatenate((np.ones(target_scores.size), np.zeros(nontarget_scores.size)))
    # Sort labels based on scores
    indices = np.argsort(all_scores, kind='mergesort')
    labels = labels[indices]
    # Compute false rejection and false acceptance rates
    tar_trial_sums = np.cumsum(labels)
    nontarget_trial_sums = nontarget_scores.size - (np.arange(1, n_scores + 1) - tar_trial_sums)
    frr = np.concatenate((np.atleast_1d(0), tar_trial_sums / target_scores.size))  # false rejection rates
    far = np.concatenate((np.atleast_1d(1), nontarget_trial_sums / nontarget_scores.size))  # false acceptance rates
    thresholds = np.concatenate((np.atleast_1d(all_scores[indices[0]] - 0.001), all_scores[indices]))  # Thresholds are the sorted scores
    return frr, far, thresholds



def compute_eer(target_scores, nontarget_scores):
    """ Returns equal error rate (EER) and the corresponding threshold. """
    frr, far, thresholds = compute_det_curve(target_scores, nontarget_scores)
    abs_diffs = np.abs(frr - far)
    min_index = np.argmin(abs_diffs)
    eer = np.mean((frr[min_index], far[min_index]))
    return eer, thresholds[min_index]


def compute_eer_auc(target_scores, nontarget_scores):
    """ Returns equal error rate (EER), AUC, and the corresponding threshold. """
    frr, far, thresholds = compute_det_curve(target_scores, nontarget_scores)
    abs_diffs = np.abs(frr - far)
    min_index = np.argmin(abs_diffs)
    eer = np.mean((frr[min_index], far[min_index]))
    
    # Compute AUC using trapezoidal rule
    auc = np.trapz(1 - frr, far)

    return eer, auc, thresholds[min_index]


# define the eval script. it should output two lists:
# # 1. list_bonafide should contain the pred scores for REAL audios
# # 2. list_spoof should contain the pred scores for FAKE audios
def eval_script(model, loader, device, savepath='B20AI056_eval', savelogs=True, printlogs=True):
    with torch.no_grad():
        model = model.to(device)
        model.eval()
        correct = 0
        total = 0
        scores = []
        truths = []
        for xs, labels in tqdm(loader):
                xs, labels = xs.to(device), labels.to(device)
                outputs = model(xs)     
                scores.extend((outputs[:, 1]).data.cpu().numpy().ravel().tolist())
                truths.extend(labels.tolist())
    
    # #calculate EER, AUC
    # target_scores = [score for score, truth in zip(scores, truths) if truth == 0]
    # nontarget_scores = [score for score, truth in zip(scores, truths) if truth == 1]
    # eer_cm, _, _ = compute_eer_auc(target_scores, nontarget_scores)

    # calculate EER, AUC correctly
    fpr, tpr, thresholds = metrics.roc_curve(np.array(truths), np.array(scores), pos_label=1)
    auc = metrics.auc(fpr, tpr)
    
    eer = brentq(lambda x : 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    thresh = interp1d(fpr, thresholds)(eer)

    if printlogs:
        tqdm.write('FINAL SCORES BELOW!!!!!!!!!!')
        tqdm.write('eer:', eer, '        auc:', auc,  "    thresh:", thresh)
        tqdm.write('FINAL SCORES ABOVE!!!!!!!!!!')
        tqdm.write()
    
    if savelogs:
        with open(os.path.join(savepath, experiment_save_name), 'w') as f:
            json.dump(
                {
                    'eer':    float(f'{eer}'    ) ,
                    'auc':    float(f'{auc}'    ) ,
                    'thresh': float(f'{thresh}' ) ,
                }, f, indent=4
            )
    eer    = float(f'{eer}'    )
    auc    = float(f'{auc}'    )
    thresh = float(f'{thresh}' )
    return eer, auc, thresh


# eval_script(model, loader, device)

# after this, start doing shit inside train.py. figure out the training and then train for 1 epoch and then run eval straight after the training and recompute everything.

def train_script(model, train_loader, eval_loader, test_loader, num_epochs, learning_rate, device, optimizer, criterion, trainloss=[], eer_train=[], eer_eval=[], eer_test=[], auc_train=[], auc_eval=[], auc_test=[], printlogs=True, savelogs=True):
    model.train()
    model = model.to(device)
    optimizer = optimizer(model.parameters(), lr=learning_rate)
    criterion = criterion()

    for epoch in range(num_epochs+1):
        if epoch > 0:
            tqdm.write(f'begun training epoch#{epoch} out of {num_epochs}')
            tqdm.write(f'-' * 20)
            bar = tqdm(total=len(train_loader))
            for xs, labels in train_loader:
                optimizer.zero_grad()
                xs, labels = xs.to(device), labels.to(device)
                outputs = model(xs)     
                scores = torch.stack([outputs[:, 1], - outputs[:, 1]], dim=1)

                # print(scores.shape, labels.shape)
                loss = criterion(scores, labels)
                loss.backward()
                optimizer.step()
                trainloss.append(loss.item())
                bar.update(1)
                bar.set_postfix({
                    'trainloss(tillnow)': np.mean(np.array(trainloss)),
                })
                wandb.log({"trainloss": loss.item()})
            bar.close()
        else:
            tqdm.write('epoch 0, printing the raw results only...')
            
        train_auc, train_eer, train_thresh = eval_script(model, train_loader, device, savelogs=False, printlogs=False)
        eval_auc, eval_eer, eval_thresh = eval_script(model, eval_loader, device, savelogs=False, printlogs=False)
        test_auc, test_eer, test_thresh = eval_script(model, test_loader, device, savelogs=False, printlogs=False)
        auc_train.append(train_auc )
        eer_train.append(train_eer )
        auc_eval.append (eval_auc  )
        eer_eval.append (eval_eer  )
        auc_test.append (test_auc  )
        eer_test.append (test_eer  )
        logs = {
            'checkpoint_save_dir': checkpoint_save_dir,
            'experiment_save_name': experiment_save_name,
            'save_location': os.path.join(checkpoint_save_dir, experiment_save_name),
            'trainloss': trainloss,
            'auc_train': auc_train,
            'eer_train': eer_train,
            'auc_eval': auc_eval,
            'eer_eval': eer_eval,
            'auc_test': auc_test,
            'eer_test': eer_test,
            'training_epochs': epoch,
            'learning_rate': learning_rate,
            'batch_size': batch_size,
        }
        
        wandb_log = {
            'auc_train' : auc_train [-1] ,
            'eer_train' : eer_train [-1] ,
            'auc_eval'  : auc_eval  [-1] ,
            'eer_eval'  : eer_eval  [-1] ,
            'auc_test'  : auc_test  [-1] ,
            'eer_test'  : eer_test  [-1] ,
        }
        
        wandb.log(wandb_log)

        if printlogs:
            print(json.dumps(obj=logs, indent=4))

        if savelogs:
            torch.save({
                'model':model.state_dict(), 
                'logs': logs
            }, os.path.join(checkpoint_save_dir, experiment_save_name + '_ckpt.pt'))
            with open(os.path.join(checkpoint_save_dir, experiment_save_name + '_logs.json'), 'w') as f:
                json.dump(logs, f, indent=4)
    return trainloss, auc_train, eer_train, auc_eval, eer_eval, auc_test, eer_test

trainloss, auc_train, eer_train, auc_eval, eer_eval, auc_test, eer_test = [], [], [], [], [], [], []
trainloss, auc_train, eer_train, auc_eval, eer_eval, auc_test, eer_test = train_script(model, train_loader, eval_loader, test_loader, nepochs, learningrate, device, torch.optim.Adam, torch.nn.CrossEntropyLoss, trainloss, eer_train, eer_eval, eer_test, auc_train, auc_eval, auc_test)

print('trainloss: ', trainloss[-1])
print('auc_train: ', auc_train[-1])
print('eer_train: ', eer_train[-1])
print('auc_eval: ', auc_eval[-1])
print('eer_eval: ', eer_eval[-1])
print('auc_test: ', auc_test[-1])
print('eer_test: ', eer_test[-1])

wandb.finish()

# threshold = -3.4
# preds = [int(score > threshold) for score in scores]
# total = sum([int(pred == pred) for (pred, truth) in zip(preds, truths)])
# correct = sum([int(pred == truth) for (pred, truth) in zip(preds, truths)])
# total, correct